<a href="https://colab.research.google.com/github/KaveeshBaddage/DataScienceImpl/blob/organize/Text_Classification-v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Import required libraries**

In [1]:
#required libraries to import dataset
import pandas as pd
import string

#required library for preprocessing
import re

#required libraries to remove stop words
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
from nltk import word_tokenize

#required libraries for Lemmatization
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

#required libraries for POS Tagging
from nltk import pos_tag
nltk.download('averaged_perceptron_tagger')

#required libraries for Named Entity Recognizion
from nltk import ne_chunk, pos_tag, word_tokenize
from nltk.tree import Tree
nltk.download('punkt')
nltk.download('maxent_ne_chunker')
nltk.download('words')

#required libraries for headword extraction
import spacy
nlp = spacy.load("en_core_web_sm")

#required libraries for TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

#required libraries for Count Vectorizer
from sklearn.feature_extraction.text import CountVectorizer

#required libraries for finalize features
from sklearn.feature_selection import SelectKBest, chi2
from scipy.sparse import coo_matrix, csr_matrix, hstack
from sklearn.preprocessing import LabelEncoder

#required libraries for SVM Model
import numpy as np
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score


#required libraries for evaluate the model
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

#required libraries for fastText
!git clone https://github.com/facebookresearch/fastText.git
!cd fastText
!pip install fastText
import fasttext.util
fasttext.util.download_model('en', if_exists='ignore')  # English
ft = fasttext.load_model('cc.en.300.bin')

# required libraries for LSTM input data preparation
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# required libraries for LSTM 
from tensorflow.keras.layers import Embedding, SpatialDropout1D, LSTM, Dense
from tensorflow.keras import Sequential


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
Cloning into 'fastText'...
remote: Enumerating objects: 3854, done.
remote: Total 3854 (delta 0), reused 0 (delta 0), pack-reused 3854
Receiving objects: 100

In [35]:
from tensorflow.keras.layers import Embedding, SpatialDropout1D, LSTM, Dense
from tensorflow.keras import Sequential


**Import Travel Dataset**

In [2]:
excel_file_url = 'https://github.com/KaveeshBaddage/DataScienceImpl/blob/main/NLP/Text%20Classification/Data/5000TravelQuestionsDataset.xlsx?raw=true'

columns = ['questions', 'coarse_classes', 'fine_classes']
dataset = pd.read_excel(excel_file_url, header=None, names=columns)
dataset['questions'].dropna(inplace=True)

In [3]:
print(dataset.info())
print(dataset['coarse_classes'].unique())
print(len(dataset['fine_classes'].unique()))


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   questions       5000 non-null   object
 1   coarse_classes  5000 non-null   object
 2   fine_classes    5000 non-null   object
dtypes: object(3)
memory usage: 117.3+ KB
None
['TTD' 'TGU' 'ACM' 'TRS' 'WTH' 'FOD' 'ENT' 'TGU\n' 'TTD\n' '\nENT']
79


# **Preprocess the dataset**

In [4]:
def preprocessText(text):


  #Converting a word to lower case
  text = text.lower()

  #remove HTML tags
  html_tags = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
  cleantext = re.sub(html_tags, '', text)

  #replace punctuation marks by space
  regular_punct = list(string.punctuation)
  for punc in regular_punct:
        if punc in text:
            text = text.replace(punc, ' ')
  
  # replace special characters by spaces
  filters='!"\'#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'
  translate_dict = dict((c, " ") for c in filters)
  translate_map = str.maketrans(translate_dict)
  text = text.translate(translate_map)

  return text;

In [5]:
# Remove HTML tags, capitalization, punctuation marks from the questions
dataset['preprocessed_questions'] = [preprocessText(question) for question in dataset['questions']]

In [6]:
#testing pre processing results
print('Sample question before preprocessing - ',dataset['questions'][1])
print('Same question after preprocessing - ',dataset['preprocessed_questions'][1])

Sample question before preprocessing -  What are the companies which organize shark feeding events for scuba divers?
Same question after preprocessing -  what are the companies which organize shark feeding events for scuba divers 


In [7]:

def removeStopwords(text):
    tokenized_words = word_tokenize(text)
    return " ".join([tokenized_word for index, tokenized_word in enumerate(tokenized_words) if (index < 1) or (tokenized_word not in stop_words)])


dataset['stopwordsRemoved_questions'] = [removeStopwords(question) for question in dataset['preprocessed_questions']]
print('Sample question before removing stop words - ',dataset['questions'][1])
print('Same question after removing its stop words - ',dataset['stopwordsRemoved_questions'][1])

Sample question before removing stop words -  What are the companies which organize shark feeding events for scuba divers?
Same question after removing its stop words -  what companies organize shark feeding events scuba divers


# **Feature Extraction**

**Lemmatization the questions**

In [8]:
 # Create WordNetLemmatizer object
wordLemmatizer = WordNetLemmatizer()

def lemmatizeText(text):
    tokenized_words = word_tokenize(text)
    lemmatized_word = [wordLemmatizer.lemmatize(word) for word in tokenized_words]
    return " ".join(lemmatized_word)


dataset['lemmatized_questions'] = [lemmatizeText(question) for question in dataset['stopwordsRemoved_questions']]
print('Sample question before lemmatization - ',dataset['questions'][1])
print('Same question after lemmatization - ',dataset['lemmatized_questions'][1])

Sample question before lemmatization -  What are the companies which organize shark feeding events for scuba divers?
Same question after lemmatization -  what company organize shark feeding event scuba diver


**POS Tagging**

POS Tagging using Hidden Markov Model Tagger

In [9]:
from nltk.tag import HiddenMarkovModelTagger
nltk.download('treebank')

# Import the toolkit and tags
import nltk
from nltk.corpus import treebank

# Train data - pretagged
train_data = treebank.tagged_sents()[:3000]

# Setup a trainer with default(None) values
# And train with the data
tagger = HiddenMarkovModelTagger.train(train_data)

def hMMposTaggedText(text):
    tokenized_words = word_tokenize(text)
    lst = [ r[1] for r in tagger.tag(tokenized_words)] 
    return ' '.join(lst)
    # posTagged_word =  [tagging.tag(word) for word in tokenized_words]
    # return ' '.join(posTagged_word)

dataset['hMMposTagged_questions'] = [hMMposTaggedText(question) for question in dataset['questions']]
print('Sample question before HMM POS Tagging - ',dataset['questions'][1])
print('Same question after HMM POS Tagging - ',dataset['hMMposTagged_questions'][1])




[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Unzipping corpora/treebank.zip.
Sample question before HMM POS Tagging -  What are the companies which organize shark feeding events for scuba divers?
Same question after HMM POS Tagging -  PRP VBP DT NNS WDT -NONE- TO VB NNS IN DT NN .


POS Tagging using NLTK pos_tag

In [10]:
def posTaggedText(text):
    tokenized_words = nltk.word_tokenize(text)
    lst = [ r[1] for r in pos_tag(tokenized_words)] 
    return ' '.join(lst)

dataset['posTagged_questions'] = [posTaggedText(question) for question in dataset['questions']]
print('Sample question before POS Tagging - ',dataset['questions'][1])
print('Same question after POS Tagging - ',dataset['posTagged_questions'][1])

Sample question before POS Tagging -  What are the companies which organize shark feeding events for scuba divers?
Same question after POS Tagging -  WP VBP DT NNS WDT VBP NN NN NNS IN NN NNS .


**Named Entity Recognition**

In [11]:

def getContinuousChunks(text):
    #chunks the text into a Tree
    chunkedText = ne_chunk(pos_tag(word_tokenize(text)))
    continuous_chunk = []
    current_chunk = []
    prev = None

    for i in chunkedText:
        if type(i) == Tree:
            current_chunk.append(" ".join([token for token, pos in i.leaves()]))
        elif current_chunk:
            named_entity = " ".join(current_chunk)
            if named_entity not in continuous_chunk:
                continuous_chunk.append(named_entity)
                current_chunk = []
        else:
            continue

    if continuous_chunk:
        named_entity = " ".join(current_chunk)
        if named_entity not in continuous_chunk:
            continuous_chunk.append(named_entity)
    
    def removeNull(continuousChunk):
        if '' in continuousChunk:
            continuousChunk.remove('')
        return continuousChunk

    newChunk = removeNull(continuous_chunk)
    return ' '.join(newChunk)


#call the function to get named entities
dataset['named_entity_questions'] = [getContinuousChunks(question) for question in dataset['questions']]
print(dataset['named_entity_questions'][1])

**Selecting as headwords**

In [12]:
# nsubjpass = A passive nominal subject is a noun phrase which is the syntactic subject of a passive clause.
# nsubj = a nominal which is the syntactic subject and the proto-agent of a clause

def head_word_tokenizer(text):
    head_words = []
    for token in nlp(text):
        if token.dep_ == "nsubj" or token.dep_ == "nsubjpass":
            head_words.append(token.text)
    return head_words

In [13]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

# Convert the text documents to a matrix of token counts
head_words_vectorizer = CountVectorizer(tokenizer = head_word_tokenizer,max_features=100,stop_words=stopwords.words('english'))
head_words_vector = head_words_vectorizer.fit_transform(dataset["questions"].values).toarray()

**Create TF-IDF Vector**

In [14]:

# build a vocabulary that only consider the top max_features ordered by term frequency across the corpus.
vectorizer = TfidfVectorizer(max_features=5000)

tfidf = vectorizer.fit_transform(dataset['lemmatized_questions'])

print(vectorizer.get_feature_names())

print(tfidf.shape)


['00pm', '10', '100', '1000', '10am', '11', '12', '1200', '124', '124th', '13', '14', '148th', '14hr', '15', '150', '16', '17', '17km', '18', '180', '19', '1st', '20', '200', '2006', '2016', '2017', '2018', '21st', '22', '23', '24', '240v', '25', '25th', '2hr', '2nd', '2wd', '30', '300', '30am', '31st', '32', '3200', '37', '3day', '3g', '3ltr', '3rd', '40', '400', '400mm', '400s', '40th', '45', '46', '4g', '4pax', '4seasons', '4wd', '4x4', '50', '500', '5am', '60nzd', '60th', '65', '68', '6km', '6pm', '6th', '700', '71', '73', '7pm', 'a1', 'a2', 'a380', 'abel', 'able', 'aboard', 'abouts', 'abq', 'abseiling', 'absolutely', 'abu', 'ac', 'academy', 'acapulco', 'accept', 'acceptable', 'accepted', 'access', 'accessed', 'accessible', 'accessory', 'accomdate', 'accommodate', 'accommodation', 'accomodation', 'account', 'accurate', 'ace', 'achievable', 'achim', 'aci', 'acitrezza', 'acquire', 'across', 'acrothea', 'active', 'activity', 'actual', 'actually', 'adagio', 'adapter', 'add', 'added', '

# Create the Bigram

In [15]:
def createBigram(data):
  #  data = nltk.word_tokenize(data)
  #  listOfBigrams = []
  #  new_words = ""
  #  bigramCounts = {}
  #  for i in range(len(data)-1):
  #     if i < len(data) - 1 and data[i+1].islower():

  #        listOfBigrams.append((data[i], data[i + 1]))

  #        if (data[i], data[i+1]) in bigramCounts:
  #           bigramCounts[(data[i], data[i + 1])] += 1
  #        else:
  #           bigramCounts[(data[i], data[i + 1])] = 1

  #  return listOfBigrams
  newWords = ""
  previousWord = None
  for word in data.strip().split(' '):
      
      if previousWord is not None:
          newWords += "{}{} ".format(previousWord, word)
      previousWord = word
  return newWords[:-1]


dataset['questions_bigram'] = [createBigram(question) for question in dataset['stopwordsRemoved_questions']]
print('Sample question before Bigram - ',dataset['questions'][1])
print('Bigram - ',dataset['questions_bigram'][1])

Sample question before Bigram -  What are the companies which organize shark feeding events for scuba divers?
Bigram -  whatcompanies companiesorganize organizeshark sharkfeeding feedingevents eventsscuba scubadivers


**Select Features**

In [16]:
x_named_entities = dataset['named_entity_questions']

x_pos_tags = dataset['posTagged_questions']

x_bigram = dataset['questions_bigram']

x_pos_tags = dataset['posTagged_questions']

x_lemma = dataset['lemmatized_questions']

x_tfigf = tfidf




In [17]:
print("x_named_entities shape",x_named_entities.shape)
print("x_pos_tags shape",x_pos_tags.shape)
print("x_bigram shape",x_bigram.shape)
print("x_pos_tags shape",x_pos_tags.shape)
print("x_lemma shape ",x_lemma.shape)

x_named_entities shape (5000,)
x_pos_tags shape (5000,)
x_bigram shape (5000,)
x_pos_tags shape (5000,)
x_lemma shape  (5000,)


# Count Vectorizer

In [18]:
# Convert a collection of text documents to a matrix of token counts
def getCountVector(document):

  # Get top 1500 max_features ordered by term frequency across the corpus.
  # Ignore terms that have a document frequency strictly higher than 0.7
  # Ignore terms that have a document frequency strictly lower than 0.5
  vectorizer = CountVectorizer(max_features=1500, min_df=5, max_df=0.7) 
  vector = vectorizer.fit_transform(document).toarray()
  return vector


# Prepare final features

In [19]:
# Stack sparse matrices horizontally
allFeatures = csr_matrix(hstack([
                        x_tfigf,
                        head_words_vector,
                        getCountVector(x_bigram),
                        getCountVector(x_pos_tags), 
                        getCountVector(x_named_entities)]))



#Encode given classes with value between 0 and (num of classes - 1)
def getEncodedClasses(className='coarse_classes'):
    encoder = LabelEncoder()
    encodedClasses = encoder.fit_transform(dataset[className])
    return encodedClasses


#Select features according to the k highest scores.
finalFeatures = SelectKBest(chi2, k=3000).fit_transform(allFeatures,getEncodedClasses('coarse_classes'))

print("Shape of the all features", allFeatures.shape)

print("Shape of the final features",finalFeatures.shape)

Shape of the all features (5000, 5905)
Shape of the final features (5000, 3000)


# **Part 1** -  Create SVM Model

In [20]:
def trainSVMModel(features, classes):
    best_prediction = None
    best_test = None
    best_accuracy = 0

    # Split dataset into 10 consecutive folds by controlling the randomness of each fold with shuffling
    cv = KFold(n_splits=10, random_state=1, shuffle=True)

    fold = 0
    accuracies = []
    for train_index, test_index in cv.split(features):
      fold += 1
      X_train, X_test = features[train_index], features[test_index]
      y_train, y_test = classes[train_index], classes[test_index]

      #Support Vector Classification 
      SVM = SVC(C=1.0, kernel='linear', degree=3, gamma='auto')

      #Fit the SVM model according to the given training data
      SVM.fit(X_train,y_train)

      #Perform classification on X_test
      predictions_SVM1 = SVM.predict(X_test)

      #computes subset accuracy
      current_accuracy = accuracy_score(predictions_SVM1, y_test)*100
      if best_accuracy < current_accuracy:
          best_accuracy = current_accuracy
          best_prediction = predictions_SVM1
          best_test = y_test
          best_model = SVM
      accuracies.append(current_accuracy)
      print("Fold {}  {}  {:.2f}".format(fold, "SVM Accuracy -> ",current_accuracy))
    
    print("Best SVM accuracy : {}".format(best_accuracy))

    #evalaute the results
    evaluateAccuracy(best_test, best_prediction)

# Evaluate Accuracy

In [21]:

def evaluateAccuracy(y_test, y_pred):
  #Calculate metrics globally by counting the total true positives, false negatives and false positives by setting average as'micro'.

  #Calculate the ratio tp / (tp + fp)
  precision = precision_score(y_test, y_pred,labels=np.unique(y_pred), average='micro') 
  print('Precision: %.2f' % (precision*100))

  #Calculate the ratio tp / (tp + fn)
  recall = recall_score(y_test, y_pred, labels=np.unique(y_pred), average='micro') 
  print('Recall: %.2f' % (recall*100))

  #Calculate the ratio 2 * (precision * recall) / (precision + recall)
  f1Score = f1_score(y_test, y_pred,labels=np.unique(y_pred), average='micro')
  print('F1 Score: %.2f' % (f1Score*100))

  #Get the confusion matrix
  cm = confusion_matrix(y_test, y_pred)
  print("\nConfustion matrix: \n{}".format(cm))

  

# Train the SVM Model and get accuracy for Coarse Classes

In [22]:
trainSVMModel(finalFeatures,getEncodedClasses('coarse_classes'))

Fold 1  SVM Accuracy ->   78.40
Fold 2  SVM Accuracy ->   79.00
Fold 3  SVM Accuracy ->   79.00
Fold 4  SVM Accuracy ->   78.60
Fold 5  SVM Accuracy ->   81.60
Fold 6  SVM Accuracy ->   82.00
Fold 7  SVM Accuracy ->   81.80
Fold 8  SVM Accuracy ->   78.60
Fold 9  SVM Accuracy ->   79.80
Fold 10  SVM Accuracy ->   77.20
Best SVM accuracy : 82.0
Precision: 82.00
Recall: 82.00
F1 Score: 82.00

Confustion matrix: 
[[ 50   0   1   3   1   4   0]
 [  1   8   0   0   0   5   0]
 [  3   0  42   1   1   3   0]
 [  4   0   3 110   9  22   0]
 [  2   1   2   3  98   4   0]
 [  3   2   3   7   1  85   0]
 [  0   0   0   1   0   0  17]]


# Train the SVM Model and get accuracy for Fine Classes

In [23]:
trainSVMModel(finalFeatures,getEncodedClasses('fine_classes'))

Fold 1  SVM Accuracy ->   52.20
Fold 2  SVM Accuracy ->   53.20
Fold 3  SVM Accuracy ->   50.40
Fold 4  SVM Accuracy ->   50.60
Fold 5  SVM Accuracy ->   52.40
Fold 6  SVM Accuracy ->   53.60
Fold 7  SVM Accuracy ->   48.80
Fold 8  SVM Accuracy ->   50.40
Fold 9  SVM Accuracy ->   54.20
Fold 10  SVM Accuracy ->   48.60
Best SVM accuracy : 54.2
Precision: 54.20
Recall: 56.93
F1 Score: 55.53

Confustion matrix: 
[[0 0 0 ... 0 0 0]
 [0 2 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 ...
 [0 0 0 ... 4 0 1]
 [0 0 0 ... 2 0 0]
 [0 0 0 ... 2 0 0]]


# **Part 2** - Word embeddings using fastText

**Word Embedding for Course classes**

In [25]:
def get_embedding_sentence(sentence):
    
    embedding=ft.get_sentence_vector(sentence)
    return embedding

In [26]:
X_doc2fast = np.array([get_embedding_sentence(question) for question in dataset['preprocessed_questions'].values])
le = LabelEncoder()
y_doc2fast = le.fit_transform(dataset['coarse_classes'])

trainSVMModel(X_doc2fast, getEncodedClasses('coarse_classes'))

Fold 1  SVM Accuracy ->   72.80
Fold 2  SVM Accuracy ->   75.60
Fold 3  SVM Accuracy ->   73.00
Fold 4  SVM Accuracy ->   72.40
Fold 5  SVM Accuracy ->   74.60
Fold 6  SVM Accuracy ->   75.20
Fold 7  SVM Accuracy ->   71.40
Fold 8  SVM Accuracy ->   71.60
Fold 9  SVM Accuracy ->   76.20
Fold 10  SVM Accuracy ->   75.20
Best SVM accuracy : 76.2
Precision: 76.20
Recall: 76.51
F1 Score: 76.35

Confustion matrix: 
[[  0   0   0   0   0   0   0   1   0]
 [  0  49   0   0  12   0   1   8   1]
 [  0   1   3   2   1   0   1   9   0]
 [  0   2   1  42   4   0   0   5   0]
 [  0   3   0   1 100   0   3  19   0]
 [  0   0   0   0   1   0   0   0   0]
 [  0   1   0   0  11   0  77   6   0]
 [  0   2   0   1  14   0   5  99   0]
 [  0   0   0   0   1   0   1   1  11]]


**Word Embedding for Fine classes**

In [27]:
trainSVMModel(X_doc2fast, getEncodedClasses('fine_classes'))

Fold 1  SVM Accuracy ->   38.80
Fold 2  SVM Accuracy ->   41.20
Fold 3  SVM Accuracy ->   41.40
Fold 4  SVM Accuracy ->   36.80
Fold 5  SVM Accuracy ->   35.60
Fold 6  SVM Accuracy ->   39.00
Fold 7  SVM Accuracy ->   34.80
Fold 8  SVM Accuracy ->   35.60
Fold 9  SVM Accuracy ->   37.60
Fold 10  SVM Accuracy ->   38.20
Best SVM accuracy : 41.4
Precision: 41.40
Recall: 58.31
F1 Score: 48.42

Confustion matrix: 
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 8 0]
 [0 0 0 ... 0 5 0]]


# **Part 3** - A NN classifier s.a. an LSTM for classification

In [29]:

# Text tokenization
tokenizer = Tokenizer(num_words=5000, split=' ')

#Updates internal vocabulary based on a list of texts.
tokenizer.fit_on_texts(dataset['questions'].values)
word_index = tokenizer.word_index
print('Number of unique tokens - %s ' % len(word_index))


# Transforms each text in questions to a sequence of integers.
initial_x = tokenizer.texts_to_sequences(dataset['questions'].values)
print('Sample equence after padding ->',initial_x[0])

# Pads sequences to the same length which equals to 25.
final_x_lstm = pad_sequences(initial_x, maxlen=25)

print('Sample sequence before padding ->',final_x_lstm[0])
print('Shape of LSTM input data tensor:', final_x_lstm.shape)





Number of unique tokens - 5582 
Sample equence after padding -> [4, 7, 2, 321, 105, 31, 1837, 17, 68, 9, 20, 71, 6, 194, 48, 32, 22, 376, 111]
Sample sequence before padding -> [   0    0    0    0    0    0    4    7    2  321  105   31 1837   17
   68    9   20   71    6  194   48   32   22  376  111]
Shape of LSTM input data tensor: (5000, 25)


In [33]:
#Convert categorical variable into dummy/indicator variables.

def convertVariables(className='coarse_classes'):
    variables = pd.get_dummies(dataset[className]).values
    return variables

**Create LSTM Model**

In [37]:
def createLSTMModel(features, classes, verbose=0):

  # Create a Sequential model
  model = Sequential()

  # Turns positive integers (indexes) into dense vectors of fixed size
  model.add(Embedding(5000, 160, input_length=features.shape[1]))

  # drops entire 1D feature maps with 0.2 rate
  model.add(SpatialDropout1D(0.2))

  #create LSTM layer
  # dimensionality of the output space is 196
  # dropout: raction of the units to drop for the linear transformation of the inputs
  # recurrent_dropout:  Fraction of the units to drop for the linear transformation of the recurrent state.
  model.add(LSTM(196, dropout=0.2, recurrent_dropout=0.2))
  model.add(Dense(classes.shape[1], activation='softmax'))
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

  if verbose == 1:
      print(model.summary())
  return model


In [38]:
createLSTMModel(final_x_lstm, convertVariables('fine_classes'), 1)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 25, 160)           800000    
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 25, 160)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 196)               279888    
_________________________________________________________________
dense_1 (Dense)              (None, 79)                15563     
Total params: 1,095,451
Trainable params: 1,095,451
Non-trainable params: 0
_________________________________________________________________
None
